# hello i suggest you to simple train process with [this code](https://www.kaggle.com/ateplyuk/pytorch-efficientnet)

i modify 

- matplotlib.image -> PIL.Image
    - so, it doesn't need 'transforms.toPIL ...'
- use torch.no_grad()
    - we need to use all. because model.eval() means 'use model layer inference mode' and torch.no_grad() means 'shut down autograd engine'. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image

import os

import torch
import torch.nn as nn
import torch.optim as optim 

import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import zipfile
with zipfile.ZipFile('../input/aerial-cactus-identification/train.zip', 'r') as zip_ref:
    zip_ref.extractall('train/')
    
with zipfile.ZipFile('../input/aerial-cactus-identification/test.zip', 'r') as zip_ref:
    zip_ref.extractall('test/')    

In [ ]:
train_dir = '/kaggle/working/train/train'
test_dir = '/kaggle/working/test/test'

train = pd.read_csv("../input/aerial-cactus-identification/train.csv")
train.head()

In [ ]:
class GetData(Dataset):
    def __init__(self, df, data_dir, transform):
        super().__init__()
        self.df = df
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):       
        img_name = self.df.id[index]
        label = self.df.has_cactus[index]
        
        img_path = os.path.join(self.data_dir, img_name)
        image = Image.open(img_path)
        image = self.transform(image)
        return image, label

In [ ]:
data_transf = transforms.Compose([
    transforms.ToTensor()
])
train_data = GetData(df = train, data_dir = train_dir, transform = data_transf)
train_loader = DataLoader(dataset = train_data, batch_size = 64,shuffle=True)

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_name('efficientnet-b1')

In [ ]:
# Unfreeze model weights
for param in model.parameters():
    param.requires_grad = True

In [ ]:
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 1)

In [ ]:
model = model.to('cuda')
optimizer = optim.Adam(model.parameters())
loss_func = nn.BCELoss()

loss_log = []

for epoch in range(5):    
    model.train()    
    for ii, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        target=target.view(-1,1).float()

        optimizer.zero_grad()
        output = model(data)                
    
        m = nn.Sigmoid()
        loss = loss_func(m(output), target)
        loss.backward()

        optimizer.step()  
        
        if ii % 1000 == 0:
            loss_log.append(loss.item())
       
    print('Epoch: {} - Loss: {:.6f}'.format(epoch + 1, loss.item()))

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(loss_log)

In [ ]:
submit = pd.read_csv('../input/aerial-cactus-identification/sample_submission.csv')
test_data = GetData(df = submit, data_dir = test_dir, transform = data_transf)
test_loader = DataLoader(dataset = test_data, shuffle=False)

In [ ]:
predict = []
with torch.no_grad():
    model.eval()
    for data,_ in test_loader:
        data = data.cuda()
        output = model(data)    

        pred = torch.sigmoid(output)
        predicted_vals = pred > 0.5
        predict.append(int(predicted_vals))
    
submit['has_cactus'] = predict
submit.to_csv('submission.csv', index=False)

In [ ]:
submit.head()

# have a good coding